# Anameka Soils Profile Extraction - Google Earth Engine

## Overview

This notebook extracts comprehensive soil and landscape data at specific coordinate points using Google Earth Engine. It retrieves topographic, soil, and landscape features from various GIS databases.

## Data Extracted

### Topographic Data
- **Elevation** (m ASL) - from USGS SRTM
- **Slope** (degrees) - derived from elevation
- **Aspect** (degrees) - derived from elevation
- **Landscape Position** - classified from topographic analysis

### Soil Properties (CSIRO SLGA)
- **Soil Depth** (cm) - depth to bedrock
- **Clay Content** (%) - at multiple depth intervals (0-5, 5-15, 15-30, 30-60, 60-100, 100-200 cm)
- **Sand Content** (%) - at multiple depth intervals
- **Silt Content** (%) - at multiple depth intervals
- **Salinity** (dS/m) - electrical conductivity

### Landscape Features
- **Granite Outcrops and Bedrock Highs** - detected from shallow soil depth
- **Soil Moisture** (m³/m³) - from ERA5-Land

## Installation

**IMPORTANT:** Run the installation cell below first if you get `ModuleNotFoundError: No module named 'ee'`

The notebook requires the `earthengine-api` package. Install it using:
```bash
pip install earthengine-api
```

Or run the installation cell below.

## 🔐 Authentication Guide (IMPORTANT - READ FIRST!)

**Before running the notebook, you MUST authenticate with Google Earth Engine:**

### Step-by-Step Authentication:

1. **Run Section 0 (Authentication Cell)** - This is the cell right after installation
   - It will automatically open a browser window
   
2. **Sign in with your Google account**
   - Use your account: **ibianchival@gmail.com**
   - Make sure you're signed in to the correct Google account
   
3. **Grant Permissions**
   - Click **"Allow"** to grant Google Earth Engine access
   - You may see a warning about the app not being verified - click **"Advanced"** then **"Go to [app name] (unsafe)"**
   
4. **Copy the Authorization Code**
   - After granting permissions, you'll see an authorization code
   - It looks like: `4/0AeanS...` (a long string)
   - **Copy this entire code**
   
5. **Paste the Code**
   - Return to the notebook
   - When prompted, **paste the authorization code**
   - Press Enter
   
6. **Verify Success**
   - You should see: "✓ Authentication complete!"
   - Then proceed to Section 1 (CONFIGURATION)

**Note:** Authentication is saved locally, so you only need to do this **once per computer**.

**Troubleshooting:**
- If browser doesn't open: Manually visit the URL shown in the output
- If code doesn't work: Make sure you copied the ENTIRE code (it's long!)
- If you see "project not found": Run Section 0 authentication first, then Section 2

## Usage Instructions

1. **Authenticate** with Google Earth Engine (Section 0 - one-time setup)
2. Set coordinates and output directory in the CONFIGURATION section (Section 1)
3. Run all cells sequentially to extract data
4. Results will be saved to the configured output directory

In [72]:
# Install Google Earth Engine API if not already installed
try:
    import ee
    print("✓ earthengine-api already installed")
except ImportError:
    print("Installing earthengine-api...")
    import subprocess
    import sys
    subprocess.check_call([sys.executable, "-m", "pip", "install", "earthengine-api", "--quiet"])
    print("✓ earthengine-api installed successfully")
    import ee

✓ earthengine-api already installed


## Section 0: Google Earth Engine Authentication

**Run this cell FIRST if you haven't authenticated before.**

This is a one-time setup step. After authentication, you can skip this section in future runs.

In [73]:
# ============================================================
# GOOGLE EARTH ENGINE AUTHENTICATION
# ============================================================
# Run this cell to authenticate with Google Earth Engine
# This is a ONE-TIME setup - authentication is saved locally

import ee

# Note: GEE_PROJECT needs to be defined first - run Section 1 (CONFIGURATION) before this
# Or define it here temporarily:
if 'GEE_PROJECT' not in globals():
    GEE_PROJECT = "ee-ibianchival"  # Default project

print("="*70)
print("GOOGLE EARTH ENGINE AUTHENTICATION")
print("="*70)
print("\nStep 1: This will open a browser window")
print("Step 2: Sign in with your Google account (ibianchival@gmail.com)")
print("Step 3: Grant permissions to Google Earth Engine")
print("Step 4: Copy the authorization code provided")
print("Step 5: Paste it in the prompt below")
print("\n" + "="*70 + "\n")

try:
    # Check if already authenticated
    ee.Initialize(project=GEE_PROJECT)
    print("✓ You are already authenticated!")
    # Try to get project ID (method varies by version)
    try:
        project_id = ee.data.getProjectId()
        print(f"✓ Current project: {project_id}")
    except AttributeError:
        # Alternative method for newer versions
        try:
            project_id = ee.Project().id().getInfo()
            print(f"✓ Current project: {project_id}")
        except:
            print(f"✓ Using project: {GEE_PROJECT}")
    print("\nYou can skip to Section 1 (CONFIGURATION) now.")
except Exception as e:
    if "no project found" in str(e).lower() or "authentication" in str(e).lower():
        print("Authentication required. Starting authentication process...\n")
        print("A browser window will open. Follow these steps:")
        print("  1. Sign in with your Google account")
        print("  2. Click 'Allow' to grant permissions")
        print("  3. Copy the authorization code")
        print("  4. Paste it below when prompted\n")
        print("-"*70)
        
        # Start authentication
        ee.Authenticate()
        
        print("\n" + "="*70)
        print("Authentication complete!")
        print("="*70)
        print("\nNow run Section 2 (GEE Setup) to verify everything works.")
    else:
        print(f"Error: {e}")
        print("\nIf authentication fails, try:")
        print("  1. Make sure you're signed in to the correct Google account")
        print("  2. Check that you have access to Google Earth Engine")
        print("  3. Try running: ee.Authenticate(auth_mode='notebook')")

GOOGLE EARTH ENGINE AUTHENTICATION

Step 1: This will open a browser window
Step 2: Sign in with your Google account (ibianchival@gmail.com)
Step 3: Grant permissions to Google Earth Engine
Step 4: Copy the authorization code provided
Step 5: Paste it in the prompt below


✓ You are already authenticated!
✓ Using project: ee-ibianchival

You can skip to Section 1 (CONFIGURATION) now.


## Section 1: CONFIGURATION

Define coordinates and output directory settings here.

In [74]:
# ============================================================
# CONFIGURATION
# ============================================================
# Update these values for your specific coordinate point

# Google Earth Engine Configuration
GEE_USER = "ibianchival@gmail.com"  # Your GEE user email
GEE_PROJECT = "ee-ibianchival"  # Your GEE project ID

# Target coordinates (decimal degrees)
LATITUDE = -31.50   # Target latitude (-90 to 90)
LONGITUDE = 117.45  # Target longitude (-180 to 180)

# Output directory for results
OUTPUT_DIR = r"C:\Users\ibian\Desktop\ClimAdapt\Anameka\-31.50_117.45_T2 North"

# Date range for soil moisture extraction (optional)
# Format: 'YYYY-MM-DD'
SOIL_MOISTURE_START_DATE = '2023-01-01'
SOIL_MOISTURE_END_DATE = '2023-12-31'

# Buffer radius around point for area statistics (meters)
# Set to 0 for point extraction only, or small value (e.g., 30m) for local statistics
BUFFER_RADIUS = 0  # meters

# Validate coordinates
if not (-90 <= LATITUDE <= 90):
    raise ValueError(f"Latitude must be between -90 and 90. Provided: {LATITUDE}")
if not (-180 <= LONGITUDE <= 180):
    raise ValueError(f"Longitude must be between -180 and 180. Provided: {LONGITUDE}")

# Create coordinate string for output files
COORDINATE_STR = f"{LATITUDE:.2f}_{LONGITUDE:.2f}"

# Ensure output directory exists
import os
os.makedirs(OUTPUT_DIR, exist_ok=True)

print("="*70)
print("CONFIGURATION")
print("="*70)
print(f"GEE User: {GEE_USER}")
print(f"GEE Project: {GEE_PROJECT if GEE_PROJECT else 'Default (auto-detect)'}")
print(f"Target Coordinate: ({LATITUDE:.6f}, {LONGITUDE:.6f})")
print(f"Output Directory: {OUTPUT_DIR}")
print(f"Soil Moisture Date Range: {SOIL_MOISTURE_START_DATE} to {SOIL_MOISTURE_END_DATE}")
print(f"Buffer Radius: {BUFFER_RADIUS} meters")
print("="*70)

CONFIGURATION
GEE User: ibianchival@gmail.com
GEE Project: ee-ibianchival
Target Coordinate: (-31.500000, 117.450000)
Output Directory: C:\Users\ibian\Desktop\ClimAdapt\Anameka\-31.50_117.45_T2 North
Soil Moisture Date Range: 2023-01-01 to 2023-12-31
Buffer Radius: 0 meters


## Section 2: Imports and Google Earth Engine Setup

In [75]:
import ee
import pandas as pd
import numpy as np
import json
from datetime import datetime
import os

# Initialize Google Earth Engine with your project
print("Initializing Google Earth Engine...")
print(f"  Project: {GEE_PROJECT}")
print(f"  User: {GEE_USER}")

try:
    # Initialize with your specific project
    ee.Initialize(project=GEE_PROJECT)
    print(f"✓ Google Earth Engine initialized successfully")
    print(f"  Using project: {GEE_PROJECT}")
except Exception as e:
    error_msg = str(e)
    print(f"✗ Initialization with project '{GEE_PROJECT}' failed")
    print(f"  Error: {error_msg}")
    
    # Check error type and provide guidance
    if "permission" in error_msg.lower() or "517222506229" in error_msg:
        print("\n  ⚠️  PROJECT PERMISSIONS ISSUE")
        print(f"  You don't have access to project '{GEE_PROJECT}'")
        print("\n  Solutions:")
        print("  1. Verify the project exists: https://code.earthengine.google.com/")
        print("  2. Make sure you're signed in as: ibianchival@gmail.com")
        print("  3. Check that 'ee-ibianchival' is listed in your projects")
        print("  4. If project doesn't exist, you can:")
        print("     - Create it in GEE Code Editor, OR")
        print("     - Set GEE_PROJECT = None in Section 1 to use default")
        print("\n  Trying default project as fallback...")
        try:
            ee.Initialize()  # Try default
            print("✓ Initialized with default project (fallback)")
            print("  Note: Some operations may still work")
        except Exception as e2:
            print(f"✗ Default project also failed: {e2}")
            raise
    elif "authentication" in error_msg.lower() or "credentials" in error_msg.lower():
        print("\n  ⚠️  AUTHENTICATION REQUIRED")
        print("  Please run Section 0 (Authentication) first.")
        raise
    else:
        print(f"\n  Unexpected error occurred")
        raise

# Verify authentication with a simple test
print("\nVerifying Google Earth Engine access...")
try:
    # Test with a simple operation using your actual coordinates (more reliable)
    # This tests with a real location instead of (0,0) which might be ocean
    test_point = ee.Geometry.Point([LONGITUDE, LATITUDE])
    test_image = ee.Image('USGS/SRTMGL1_003').select('elevation')
    print(f"  Testing data access at your coordinates ({LATITUDE:.2f}, {LONGITUDE:.2f})...")
    
    # Use reduceRegion instead of sample for more reliable results
    test_result = test_image.reduceRegion(
        reducer=ee.Reducer.first(),
        geometry=test_point,
        scale=30,
        bestEffort=True
    ).getInfo()
    
    if 'elevation' in test_result and test_result['elevation'] is not None:
        test_value = test_result['elevation']
        print(f"  ✓ Test successful (elevation: {test_value:.1f} m)")
        print("✓ Google Earth Engine authentication verified")
    else:
        print("  ⚠️  Test completed but no elevation data at test point")
        print("  ✓ GEE connection is working (this is normal for some locations)")
        print("✓ Google Earth Engine authentication verified")
        
except Exception as e:
    error_msg = str(e)
    print(f"\n✗ Verification test failed")
    print(f"  Error: {error_msg}")
    
    # Provide specific guidance
    if "permission" in error_msg.lower() or "517222506229" in error_msg:
        print("\n  ⚠️  PROJECT PERMISSIONS ISSUE")
        print("  GEE is trying to use a project you don't have access to.")
        print("\n  Solutions:")
        print("  1. Visit: https://code.earthengine.google.com/")
        print("  2. Check your projects list")
        print("  3. Verify project 'ee-ibianchival' exists")
        print("\n  You can try to continue, but operations may fail.")
    elif "authentication" in error_msg.lower():
        print("\n  ⚠️  Authentication issue - please run Section 0 first")
    elif "null" in error_msg.lower() or "required" in error_msg.lower():
        print("\n  ⚠️  Test operation issue (may be normal)")
        print("  The test point may not have data, but GEE connection is working.")
        print("  This is OK - you can proceed with data extraction.")
    else:
        print("\n  Note: GEE initialized successfully, but test had issues.")
        print("  This may be normal - you can try proceeding with data extraction.")
    
    # Don't raise - initialization succeeded, test is just a check
    print("\n  ✓ GEE initialized successfully - proceeding...")

print("\n✓ Libraries imported successfully")

Initializing Google Earth Engine...
  Project: ee-ibianchival
  User: ibianchival@gmail.com
✓ Google Earth Engine initialized successfully
  Using project: ee-ibianchival

Verifying Google Earth Engine access...
  Testing data access at your coordinates (-31.50, 117.45)...
  ✓ Test successful (elevation: 234.0 m)
✓ Google Earth Engine authentication verified

✓ Libraries imported successfully


## Section 3: Point Feature Creation

In [76]:
# Create point geometry from coordinates
point = ee.Geometry.Point([LONGITUDE, LATITUDE])

# Create buffer if needed for area statistics
if BUFFER_RADIUS > 0:
    point_buffer = point.buffer(BUFFER_RADIUS)
    print(f"Created point with {BUFFER_RADIUS}m buffer for area statistics")
else:
    point_buffer = point
    print("Using point extraction (no buffer)")

print(f"Point coordinates: ({LATITUDE:.6f}, {LONGITUDE:.6f})")

Using point extraction (no buffer)
Point coordinates: (-31.500000, 117.450000)


## Section 4: Topographic Data Extraction

Extract elevation, slope, aspect, and calculate Topographic Wetness Index (TWI) for landscape position classification.

In [77]:
def extract_topographic_data(point_geom, buffer_geom):
    """
    Extract topographic data: elevation, slope, aspect, and TWI.
    
    Returns:
        dict: Dictionary with topographic values
    """
    print("\nExtracting topographic data...")
    
    # Load SRTM elevation data
    dem = ee.Image('USGS/SRTMGL1_003').select('elevation')
    
    # Calculate slope and aspect
    slope = ee.Terrain.slope(dem)
    aspect = ee.Terrain.aspect(dem)
    
    # Calculate Topographic Wetness Index (TWI) for landscape position
    # TWI = ln(flow_accumulation / tan(slope))
    # Simplified version using focal mean as flow accumulation proxy
    flow_accum_proxy = slope.focal_mean(radius=100, units='meters').multiply(-1).add(90)
    slope_rad = slope.multiply(np.pi).divide(180)
    tan_slope = slope_rad.tan().max(0.001)
    twi = flow_accum_proxy.log().subtract(tan_slope.log()).rename('TWI')
    
    # Combine all topographic layers
    topographic_image = dem.addBands([slope, aspect, twi])
    
    # Always calculate elevation min/max over 1km radius
    analysis_area = point_geom.buffer(1000)
    
    # Extract values at point
    if BUFFER_RADIUS > 0:
        # Use reduceRegion for area statistics
        stats = topographic_image.reduceRegion(
            reducer=ee.Reducer.mean().combine(
                reducer2=ee.Reducer.minMax(),
                outputPrefix='',
                sharedInputs=True
            ),
            geometry=buffer_geom,
            scale=30,
            bestEffort=True,
            maxPixels=1e9
        )
        result = stats.getInfo()
        
        # Extract mean values
        elevation = result.get('elevation_mean', None)
        slope_val = result.get('slope_mean', None)
        aspect_val = result.get('aspect_mean', None)
        twi_val = result.get('TWI_mean', None)
        
        print(f"  Elevation: {elevation:.2f} m")
        print(f"  Slope: {slope_val:.2f} degrees")
        print(f"  Aspect: {aspect_val:.2f} degrees")
        print(f"  TWI: {twi_val:.2f}")
    else:
        # Use sample for point extraction
        sample = topographic_image.sample(point_geom, scale=30).first()
        result = sample.getInfo()['properties']
        
        elevation = result.get('elevation', None)
        slope_val = result.get('slope', None)
        aspect_val = result.get('aspect', None)
        twi_val = result.get('TWI', None)
        
        print(f"  Elevation: {elevation:.2f} m")
        print(f"  Slope: {slope_val:.2f} degrees")
        print(f"  Aspect: {aspect_val:.2f} degrees")
        print(f"  TWI: {twi_val:.2f}")
    
    # Calculate elevation min/max over 1km radius
    elev_stats_1km = dem.reduceRegion(
        reducer=ee.Reducer.minMax(),
        geometry=analysis_area,
        scale=30,
        bestEffort=True,
        maxPixels=1e9
    ).getInfo()
    
    elevation_min_1km = elev_stats_1km.get('elevation_min', elevation)
    elevation_max_1km = elev_stats_1km.get('elevation_max', elevation)
    
    print(f"  Elevation range (1km radius): {elevation_min_1km:.2f} - {elevation_max_1km:.2f} m")
    
    return {
        'elevation_m': elevation,
        'elevation_min_1km_m': elevation_min_1km,
        'elevation_max_1km_m': elevation_max_1km,
        'slope_degrees': slope_val,
        'aspect_degrees': aspect_val,
        'twi': twi_val
    }

# Extract topographic data
topographic_data = extract_topographic_data(point, point_buffer)
print("✓ Topographic data extracted")


Extracting topographic data...
  Elevation: 234.00 m
  Slope: 4.28 degrees
  Aspect: 130.45 degrees
  TWI: 7.06
  Elevation range (1km radius): 224.00 - 253.00 m
✓ Topographic data extracted


## Section 5: Soil Data Extraction (CSIRO SLGA)

Extract soil properties from CSIRO Soil and Landscape Grid of Australia (SLGA) dataset.

In [ ]:
def extract_soil_properties(point_geom, buffer_geom):
    """
    Extract soil properties from CSIRO SLGA dataset.
    
    Returns:
        dict: Dictionary with all soil property values
    """
    print("\nExtracting soil properties from CSIRO SLGA...")
    
    # Load SLGA collection
    slga = ee.ImageCollection('CSIRO/SLGA')
    
    # Depth bands available in SLGA
    depth_bands = [
        '000_005',  # 0-5 cm
        '005_015',  # 5-15 cm
        '015_030',  # 15-30 cm
        '030_060',  # 30-60 cm
        '060_100',  # 60-100 cm
        '100_200'   # 100-200 cm
    ]
    
    # Attributes to extract
    attributes = {
        'CLY': 'clay',      # Clay content (%)
        'SND': 'sand',      # Sand content (%)
        'SLT': 'silt',      # Silt content (%)
        'DES': 'soil_depth', # Depth of soil (cm)
        'ECE': 'salinity'   # Electrical conductivity (dS/m) - salinity
    }
    
    soil_data = {}
    
    # Extract each attribute
    for attr_code, attr_name in attributes.items():
        print(f"\n  Processing {attr_name} ({attr_code})...")
        
        # Filter collection by attribute code
        attr_collection = slga.filter(ee.Filter.eq('attribute_code', attr_code))
        
        if attr_collection.size().getInfo() == 0:
            print(f"    Warning: No data found for {attr_code}")
            continue
        
        # Get the first image (SLGA typically has one image per attribute)
        attr_image = attr_collection.first()
        
        # Get all band names
        band_names = attr_image.bandNames().getInfo()
        print(f"    Available bands: {band_names}")
        
        # Extract values for each depth band
        for depth in depth_bands:
            # Look for band matching depth pattern (e.g., CLY_000_005_EV)
            matching_bands = [b for b in band_names if depth in b]
            
            if not matching_bands:
                continue
            
            band_name = matching_bands[0]  # Use first matching band
            
            # Select the band
            band_image = attr_image.select(band_name)
            
            # Extract value
            try:
                if BUFFER_RADIUS > 0:
                    stats = band_image.reduceRegion(
                        reducer=ee.Reducer.mean(),
                        geometry=buffer_geom,
                        scale=90,  # SLGA resolution is 90m
                        bestEffort=True,
                        maxPixels=1e9
                    )
                    value = stats.get(band_name).getInfo()
                else:
                    sample = band_image.sample(point_geom, scale=90).first()
                    value = sample.getInfo()['properties'].get(band_name, None)
                
                if value is not None:
                    # For DES (soil depth), convert from normalized 0-1 to cm (0-200cm)
                    if attr_code == 'DES':
                        value = value * 200
                    
                    # Store value
                    key = f"{attr_name}_{depth.replace('_', '-')}cm"
                    soil_data[key] = value
                    print(f"      {key}: {value:.2f}")
            except Exception as e:
                print(f"      Error extracting {band_name}: {e}")
                continue
    
    return soil_data

# Extract soil properties
soil_data = extract_soil_properties(point, point_buffer)
print("\n✓ Soil properties extracted")


Extracting soil properties from CSIRO SLGA...

  Processing clay (CLY)...
    Available bands: ['CLY_000_005_EV', 'CLY_000_005_05', 'CLY_000_005_95', 'CLY_005_015_EV', 'CLY_005_015_05', 'CLY_005_015_95', 'CLY_015_030_EV', 'CLY_015_030_05', 'CLY_015_030_95', 'CLY_030_060_EV', 'CLY_030_060_05', 'CLY_030_060_95', 'CLY_060_100_EV', 'CLY_060_100_05', 'CLY_060_100_95', 'CLY_100_200_EV', 'CLY_100_200_05', 'CLY_100_200_95']
      clay_000-005cm: 13.52
      clay_005-015cm: 15.93
      clay_015-030cm: 24.36
      clay_030-060cm: 33.33
      clay_060-100cm: 37.02
      clay_100-200cm: 34.58

  Processing sand (SND)...
    Available bands: ['SND_000_005_EV', 'SND_000_005_05', 'SND_000_005_95', 'SND_005_015_EV', 'SND_005_015_05', 'SND_005_015_95', 'SND_015_030_EV', 'SND_015_030_05', 'SND_015_030_95', 'SND_030_060_EV', 'SND_030_060_05', 'SND_030_060_95', 'SND_060_100_EV', 'SND_060_100_05', 'SND_060_100_95', 'SND_100_200_EV', 'SND_100_200_05', 'SND_100_200_95']
      sand_000-005cm: 80.20


## Section 6: Landscape Position Classification

Classify landscape position based on topographic analysis (TWI and relative elevation).

In [ ]:
def classify_landscape_position(elevation, slope, aspect, twi, point_geom, buffer_geom=None):
    """
    Classify landscape position based on topographic metrics.
    
    Returns:
        dict: Landscape position classification and metrics
    """
    print("\nClassifying landscape position...")
    
    # Load DEM for relative elevation calculation
    dem = ee.Image('USGS/SRTMGL1_003').select('elevation')
    
    # Calculate relative elevation within local area (1km radius)
    if buffer_geom is not None:
        local_area = buffer_geom
    else:
        local_area = point_geom.buffer(1000)
    
    # Get elevation statistics for local area
    # Fix: combine() uses keyword arguments, not a dictionary
    elev_stats = dem.reduceRegion(
        reducer=ee.Reducer.minMax().combine(
            reducer2=ee.Reducer.mean(),
            outputPrefix='',
            sharedInputs=True
        ),
        geometry=local_area,
        scale=30,
        bestEffort=True,
        maxPixels=1e9
    ).getInfo()
    
    elev_min = elev_stats.get('elevation_min', elevation)
    elev_max = elev_stats.get('elevation_max', elevation)
    elev_mean = elev_stats.get('elevation_mean', elevation)
    
    # Calculate relative elevation (0 = lowest, 1 = highest) for classification only
    if elev_max > elev_min:
        relative_elevation = (elevation - elev_min) / (elev_max - elev_min)
    else:
        relative_elevation = 0.5
    
    print(f"  Elevation range (1km radius): {elev_min:.2f} - {elev_max:.2f} m")
    print(f"  TWI: {twi:.2f}")
    print(f"  Slope: {slope:.2f} degrees")
    
    # Classify based on TWI and relative elevation
    if relative_elevation > 0.8 and twi < 5:
        position = "Ridge"
    elif relative_elevation > 0.6 and twi < 8:
        position = "Upper Slope"
    elif relative_elevation > 0.4 and twi < 12:
        position = "Mid Slope"
    elif relative_elevation > 0.2 and twi < 15:
        position = "Lower Slope"
    elif relative_elevation < 0.2 or twi >= 15:
        position = "Valley"
    else:
        if twi < 5:
            position = "Upper Slope"
        elif twi < 8:
            position = "Mid Slope"
        elif twi < 12:
            position = "Lower Slope"
        else:
            position = "Valley"
    
    print(f"  Landscape Position: {position}")
    
    return {
        'landscape_position': position,
        'local_elevation_min': elev_min,
        'local_elevation_max': elev_max,
        'local_elevation_mean': elev_mean
    }

# Classify landscape position
landscape_data = classify_landscape_position(
    topographic_data['elevation_m'],
    topographic_data['slope_degrees'],
    topographic_data['aspect_degrees'],
    topographic_data['twi'],
    point,
    point_buffer
)
print("✓ Landscape position classified")


Classifying landscape position...
  Elevation range (1km radius): 257.00 - 257.00 m
  TWI: 8.61
  Slope: 0.93 degrees
  Landscape Position: Mid Slope
✓ Landscape position classified


## Section 7: Granite Outcrops and Bedrock Highs Detection

Detect granite outcrops and bedrock highs based on shallow soil depth.

In [ ]:
def detect_rock_outcrops(point_geom, buffer_geom):
    """Detect granite outcrops and bedrock highs based on shallow soil depth."""
    print("\nDetecting granite outcrops and bedrock highs...")
    
    slga = ee.ImageCollection('CSIRO/SLGA')
    des_collection = slga.filter(ee.Filter.eq('attribute_code', 'DES'))
    
    if des_collection.size().getInfo() == 0:
        print("  Warning: Soil depth data (DES) not available")
        return {'rock_outcrop_probability': None, 'bedrock_high': None, 'soil_depth_cm': None, 'soil_depth_min_cm': None, 'soil_depth_max_cm': None, 'rock_classification': 'Unknown'}
    
    des_image = des_collection.first()
    band_names = des_image.bandNames().getInfo()
    
    depth_band = None
    for band in band_names:
        if '000_200' in band or '0_200' in band:
            depth_band = band
            break
    
    if not depth_band:
        depth_band = band_names[0] if band_names else None
    
    if not depth_band:
        return {'rock_outcrop_probability': None, 'bedrock_high': None, 'soil_depth_cm': None, 'soil_depth_min_cm': None, 'soil_depth_max_cm': None, 'rock_classification': 'Unknown'}
    
    depth_image = des_image.select(depth_band)
    
    # Create 1km radius buffer for analysis
    analysis_area = point_geom.buffer(1000)
    
    # Calculate soil depth statistics over 1km radius
    stats = depth_image.reduceRegion(
        reducer=ee.Reducer.minMax().combine(
            reducer2=ee.Reducer.mean(),
            outputPrefix='',
            sharedInputs=True
        ),
        geometry=analysis_area,
        scale=90,
        bestEffort=True,
        maxPixels=1e9
    ).getInfo()
    
    depth_mean_normalized = stats.get(f'{depth_band}_mean', None)
    depth_min_normalized = stats.get(f'{depth_band}_min', None)
    depth_max_normalized = stats.get(f'{depth_band}_max', None)
    
    if depth_mean_normalized is None:
        return {'rock_outcrop_probability': None, 'bedrock_high': None, 'soil_depth_cm': None, 'soil_depth_min_cm': None, 'soil_depth_max_cm': None, 'rock_classification': 'Unknown'}
    
    # Convert normalized values (0-1) to cm (0-200cm)
    soil_depth_cm = depth_mean_normalized * 200
    soil_depth_min_cm = depth_min_normalized * 200 if depth_min_normalized is not None else None
    soil_depth_max_cm = depth_max_normalized * 200 if depth_max_normalized is not None else None
    
    print(f"  Soil depth (mean): {soil_depth_cm:.1f} cm")
    if soil_depth_min_cm is not None and soil_depth_max_cm is not None:
        print(f"  Soil depth range (1km radius): {soil_depth_min_cm:.1f} - {soil_depth_max_cm:.1f} cm")
    
    # Calculate rock outcrop probability based on mean soil depth
    if soil_depth_cm < 15:
        bedrock_high = True
        rock_outcrop_probability = 1.0
        classification = "Bedrock High"
    elif soil_depth_cm < 30:
        bedrock_high = False
        rock_outcrop_probability = 0.7 + (30 - soil_depth_cm) / 30 * 0.3
        classification = "Rock Outcrop"
    else:
        bedrock_high = False
        rock_outcrop_probability = max(0, 1.0 - (soil_depth_cm - 30) / 100)
        classification = "Soil Present"
    
    print(f"  Classification: {classification}")
    print(f"  Rock outcrop probability (1km radius): {rock_outcrop_probability:.2f}")
    
    return {
        'rock_outcrop_probability': rock_outcrop_probability,
        'bedrock_high': bedrock_high,
        'soil_depth_cm': soil_depth_cm,
        'soil_depth_min_cm': soil_depth_min_cm,
        'soil_depth_max_cm': soil_depth_max_cm,
        'rock_classification': classification
    }

rock_data = detect_rock_outcrops(point, point_buffer)
print("✓ Rock outcrop detection completed")


Detecting granite outcrops and bedrock highs...
  Soil depth (mean): 148.3 cm
  Soil depth range (1km radius): 132.5 - 169.6 cm
  Classification: Soil Present
  Rock outcrop probability (1km radius): 0.00
✓ Rock outcrop detection completed


## Section 8: Soil Moisture Extraction

Extract soil moisture data from ERA5-Land dataset.

In [ ]:
def extract_soil_moisture(point_geom, buffer_geom, start_date, end_date):
    """Extract soil moisture temporal dynamics from ERA5-Land monthly aggregated data."""
    print("\nExtracting soil moisture temporal dynamics from ERA5-Land...")
    print(f"  Date range: {start_date} to {end_date}")
    
    try:
        era5 = ee.ImageCollection('ECMWF/ERA5_LAND/MONTHLY_AGGR').filterDate(start_date, end_date)
        
        collection_size = era5.size().getInfo()
        if collection_size == 0:
            return {
                'soil_moisture_surface_m3m3': None, 
                'soil_moisture_rootzone_m3m3': None, 
                'soil_moisture_mean_m3m3': None,
                'soil_moisture_monthly': {}
            }
        
        print(f"  Found {collection_size} monthly images")
        
        # Get list of images with dates
        image_list = era5.toList(collection_size)
        
        # Extract monthly time series
        monthly_data = []
        surface_values = []
        rootzone_values = []
        mean_values = []
        
        for i in range(collection_size):
            image = ee.Image(image_list.get(i))
            
            # Get image date
            image_date = ee.Date(image.get('system:time_start'))
            date_str = image_date.format('YYYY-MM').getInfo()
            
            # Extract surface and rootzone moisture
            surface_layer = image.select('volumetric_soil_water_layer_1')
            rootzone_layer2 = image.select('volumetric_soil_water_layer_2')
            rootzone_layer3 = image.select('volumetric_soil_water_layer_3')
            rootzone_layer = rootzone_layer2.add(rootzone_layer3).divide(2)
            
            # Combine layers
            moisture_image = surface_layer.addBands([rootzone_layer])
            
            # Extract values
            if BUFFER_RADIUS > 0:
                stats = moisture_image.reduceRegion(
                    reducer=ee.Reducer.mean(), 
                    geometry=buffer_geom, 
                    scale=11132, 
                    bestEffort=True, 
                    maxPixels=1e9
                )
                result = stats.getInfo()
                surface_val = result.get('volumetric_soil_water_layer_1', None)
                rootzone_val = result.get('volumetric_soil_water_layer_2', None)
            else:
                sample = moisture_image.sample(point_geom, scale=11132).first()
                props = sample.getInfo()['properties']
                surface_val = props.get('volumetric_soil_water_layer_1', None)
                rootzone_val = props.get('volumetric_soil_water_layer_2', None)
            
            if surface_val is not None and rootzone_val is not None:
                mean_val = (surface_val + rootzone_val) / 2
                monthly_data.append({
                    'date': date_str,
                    'surface_m3m3': surface_val,
                    'rootzone_m3m3': rootzone_val,
                    'mean_m3m3': mean_val
                })
                surface_values.append(surface_val)
                rootzone_values.append(rootzone_val)
                mean_values.append(mean_val)
        
        # Calculate summary statistics
        if surface_values:
            surface_mean = np.mean(surface_values)
            surface_min = np.min(surface_values)
            surface_max = np.max(surface_values)
            surface_std = np.std(surface_values)
            
            rootzone_mean = np.mean(rootzone_values)
            rootzone_min = np.min(rootzone_values)
            rootzone_max = np.max(rootzone_values)
            rootzone_std = np.std(rootzone_values)
            
            mean_moisture = np.mean(mean_values)
            mean_min = np.min(mean_values)
            mean_max = np.max(mean_values)
            mean_std = np.std(mean_values)
            
            print(f"  Monthly time series extracted: {len(monthly_data)} months")
            print(f"  Surface moisture (0-7cm): mean={surface_mean:.4f}, min={surface_min:.4f}, max={surface_max:.4f} m³/m³")
            print(f"  Root zone moisture (7-100cm): mean={rootzone_mean:.4f}, min={rootzone_min:.4f}, max={rootzone_max:.4f} m³/m³")
            print(f"  Mean moisture: mean={mean_moisture:.4f}, min={mean_min:.4f}, max={mean_max:.4f} m³/m³")
            
            # Create monthly dictionary for export
            monthly_dict = {}
            for month_data in monthly_data:
                date_key = month_data['date']
                monthly_dict[f'soil_moisture_surface_{date_key}_m3m3'] = month_data['surface_m3m3']
                monthly_dict[f'soil_moisture_rootzone_{date_key}_m3m3'] = month_data['rootzone_m3m3']
                monthly_dict[f'soil_moisture_mean_{date_key}_m3m3'] = month_data['mean_m3m3']
            
            return {
                'soil_moisture_surface_m3m3': surface_mean,
                'soil_moisture_surface_min_m3m3': surface_min,
                'soil_moisture_surface_max_m3m3': surface_max,
                'soil_moisture_surface_std_m3m3': surface_std,
                'soil_moisture_rootzone_m3m3': rootzone_mean,
                'soil_moisture_rootzone_min_m3m3': rootzone_min,
                'soil_moisture_rootzone_max_m3m3': rootzone_max,
                'soil_moisture_rootzone_std_m3m3': rootzone_std,
                'soil_moisture_mean_m3m3': mean_moisture,
                'soil_moisture_mean_min_m3m3': mean_min,
                'soil_moisture_mean_max_m3m3': mean_max,
                'soil_moisture_mean_std_m3m3': mean_std,
                'soil_moisture_monthly': monthly_dict
            }
        else:
            return {
                'soil_moisture_surface_m3m3': None, 
                'soil_moisture_rootzone_m3m3': None, 
                'soil_moisture_mean_m3m3': None,
                'soil_moisture_monthly': {}
            }
            
    except Exception as e:
        print(f"  Error: {e}")
        import traceback
        traceback.print_exc()
        return {
            'soil_moisture_surface_m3m3': None, 
            'soil_moisture_rootzone_m3m3': None, 
            'soil_moisture_mean_m3m3': None,
            'soil_moisture_monthly': {}
        }

soil_moisture_data = extract_soil_moisture(point, point_buffer, SOIL_MOISTURE_START_DATE, SOIL_MOISTURE_END_DATE)
print("✓ Soil moisture extracted")


Extracting soil moisture from ERA5-Land...
  Date range: 2023-01-01 to 2023-12-31
  Surface moisture (0-7cm): 0.1856 m³/m³
  Root zone moisture (7-100cm): 0.1647 m³/m³
  Mean moisture: 0.1751 m³/m³
✓ Soil moisture extracted


## Section 9: Data Compilation and Export

Combine all extracted data and export to CSV and JSON files.

In [ ]:
def compile_and_export_results(coordinate_str, latitude, longitude, topographic_data, soil_data, landscape_data, rock_data, soil_moisture_data, output_dir):
    """Compile all extracted data into two-column format (Column A: titles, Column B: values) and export to files."""
    print("\n" + "="*70)
    print("Compiling Results")
    print("="*70)
    
    # Remove relative_elevation from landscape_data if present
    landscape_data_clean = {k: v for k, v in landscape_data.items() if k != 'relative_elevation'}
    
    # Build a flat dictionary with all data
    all_data = {}
    
    # Basic information
    all_data['point_id'] = coordinate_str
    all_data['latitude'] = latitude
    all_data['longitude'] = longitude
    all_data['extraction_date'] = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    # Topographic data
    all_data['elevation_m'] = topographic_data.get('elevation_m')
    all_data['elevation_min_1km_m'] = topographic_data.get('elevation_min_1km_m')
    all_data['elevation_max_1km_m'] = topographic_data.get('elevation_max_1km_m')
    all_data['slope_degrees'] = topographic_data.get('slope_degrees')
    all_data['aspect_degrees'] = topographic_data.get('aspect_degrees')
    all_data['twi'] = topographic_data.get('twi')
    
    # Landscape data (without relative_elevation)
    all_data['landscape_position'] = landscape_data_clean.get('landscape_position')
    all_data['local_elevation_min'] = landscape_data_clean.get('local_elevation_min')
    all_data['local_elevation_max'] = landscape_data_clean.get('local_elevation_max')
    all_data['local_elevation_mean'] = landscape_data_clean.get('local_elevation_mean')
    
    # Rock data
    all_data['rock_outcrop_probability'] = rock_data.get('rock_outcrop_probability')
    all_data['bedrock_high'] = rock_data.get('bedrock_high')
    all_data['soil_depth_cm'] = rock_data.get('soil_depth_cm')
    all_data['soil_depth_min_cm'] = rock_data.get('soil_depth_min_cm')
    all_data['soil_depth_max_cm'] = rock_data.get('soil_depth_max_cm')
    all_data['rock_classification'] = rock_data.get('rock_classification')
    
    # Soil moisture data - summary statistics
    all_data['soil_moisture_surface_m3m3'] = soil_moisture_data.get('soil_moisture_surface_m3m3')
    all_data['soil_moisture_surface_min_m3m3'] = soil_moisture_data.get('soil_moisture_surface_min_m3m3')
    all_data['soil_moisture_surface_max_m3m3'] = soil_moisture_data.get('soil_moisture_surface_max_m3m3')
    all_data['soil_moisture_surface_std_m3m3'] = soil_moisture_data.get('soil_moisture_surface_std_m3m3')
    all_data['soil_moisture_rootzone_m3m3'] = soil_moisture_data.get('soil_moisture_rootzone_m3m3')
    all_data['soil_moisture_rootzone_min_m3m3'] = soil_moisture_data.get('soil_moisture_rootzone_min_m3m3')
    all_data['soil_moisture_rootzone_max_m3m3'] = soil_moisture_data.get('soil_moisture_rootzone_max_m3m3')
    all_data['soil_moisture_rootzone_std_m3m3'] = soil_moisture_data.get('soil_moisture_rootzone_std_m3m3')
    all_data['soil_moisture_mean_m3m3'] = soil_moisture_data.get('soil_moisture_mean_m3m3')
    all_data['soil_moisture_mean_min_m3m3'] = soil_moisture_data.get('soil_moisture_mean_min_m3m3')
    all_data['soil_moisture_mean_max_m3m3'] = soil_moisture_data.get('soil_moisture_mean_max_m3m3')
    all_data['soil_moisture_mean_std_m3m3'] = soil_moisture_data.get('soil_moisture_mean_std_m3m3')
    
    # Soil moisture monthly time series (temporal dynamics)
    monthly_moisture = soil_moisture_data.get('soil_moisture_monthly', {})
    if monthly_moisture:
        # Sort monthly data by date for consistent ordering
        sorted_monthly = sorted(monthly_moisture.items())
        for key, value in sorted_monthly:
            all_data[key] = value
    
    # Soil properties (already in correct format: clay_000-005cm, sand_000-005cm, etc.)
    for key, value in soil_data.items():
        all_data[key] = value
    
    # Create two-column DataFrame: Column A = titles, Column B = values
    rows = []
    for title, value in all_data.items():
        # Format value for display
        if pd.isna(value):
            formatted_value = None
        elif isinstance(value, bool):
            formatted_value = value
        elif isinstance(value, (int, float, np.integer, np.floating)):
            formatted_value = value
        else:
            formatted_value = value
        
        rows.append({'Data_Title': title, 'Value': formatted_value})
    
    df = pd.DataFrame(rows)
    
    # Create Excel file with multiple sheets
    excel_filename = f"{coordinate_str}_soils_profile.xlsx"
    excel_path = os.path.join(output_dir, excel_filename)
    
    with pd.ExcelWriter(excel_path, engine='openpyxl') as writer:
        # Sheet 1: All data (two-column format)
        df.to_excel(writer, sheet_name='All Data', index=False, float_format='%.4f')
        
        # Sheet 2: Soil Composition by Depth Intervals
        depth_interval_rows = []
        
        # Define depth interval mappings (as shown in the image)
        depth_intervals = [
            {
                'interval': '0 - 30cm',
                'clay_key': 'clay_015-030cm',
                'sand_key': 'sand_015-030cm',
                'silt_key': 'silt_015-030cm'
            },
            {
                'interval': '30 - 60cm',
                'clay_key': 'clay_030-060cm',
                'sand_key': 'sand_030-060cm',
                'silt_key': 'silt_030-060cm'
            },
            {
                'interval': '60 - 100cm',
                'clay_key': 'clay_060-100cm',
                'sand_key': 'sand_060-100cm',
                'silt_key': 'silt_060-100cm'
            }
        ]
        
        for depth_info in depth_intervals:
            # Add depth interval header
            depth_interval_rows.append({
                'Soil Type/Depth': depth_info['interval'],
                'Value': None
            })
            
            # Add clay, sand, silt values
            clay_val = soil_data.get(depth_info['clay_key'])
            sand_val = soil_data.get(depth_info['sand_key'])
            silt_val = soil_data.get(depth_info['silt_key'])
            
            if clay_val is not None:
                depth_interval_rows.append({
                    'Soil Type/Depth': depth_info['clay_key'],
                    'Value': clay_val
                })
            
            if sand_val is not None:
                depth_interval_rows.append({
                    'Soil Type/Depth': depth_info['sand_key'],
                    'Value': sand_val
                })
            
            if silt_val is not None:
                depth_interval_rows.append({
                    'Soil Type/Depth': depth_info['silt_key'],
                    'Value': silt_val
                })
        
        df_depth_intervals = pd.DataFrame(depth_interval_rows)
        df_depth_intervals.to_excel(writer, sheet_name='Soil Composition', index=False, float_format='%.5f')
    
    print(f"\n✓ Excel file saved: {excel_filename}")
    print(f"  Sheet 1 'All Data': 2 columns (Data_Title, Value), {len(df)} rows")
    print(f"  Sheet 2 'Soil Composition': Depth intervals grouped table, {len(df_depth_intervals)} rows")
    
    # Also save CSV for backward compatibility
    csv_filename = f"{coordinate_str}_soils_profile.csv"
    csv_path = os.path.join(output_dir, csv_filename)
    df.to_csv(csv_path, index=False, float_format='%.4f')
    print(f"✓ CSV file saved: {csv_filename}")
    
    json_filename = f"{coordinate_str}_soils_profile.json"
    json_path = os.path.join(output_dir, json_filename)
    
    # For JSON, create a dictionary format
    json_data = {}
    for title, value in all_data.items():
        if pd.isna(value):
            json_data[title] = None
        elif isinstance(value, (np.integer, np.floating)):
            json_data[title] = float(value)
        elif isinstance(value, bool):
            json_data[title] = value
        else:
            json_data[title] = value
    
    with open(json_path, 'w', encoding='utf-8') as f:
        json.dump(json_data, f, indent=2, ensure_ascii=False)
    
    print(f"✓ JSON file saved: {json_filename}")
    
    print("\n" + "="*70)
    print("Extraction Summary")
    print("="*70)
    print(f"Point: ({latitude:.6f}, {longitude:.6f})")
    print(f"\nTopographic:")
    print(f"  Elevation: {topographic_data['elevation_m']:.2f} m")
    print(f"  Elevation range (1km): {topographic_data.get('elevation_min_1km_m', 'N/A'):.2f} - {topographic_data.get('elevation_max_1km_m', 'N/A'):.2f} m")
    print(f"  Slope: {topographic_data['slope_degrees']:.2f}°")
    print(f"  Aspect: {topographic_data['aspect_degrees']:.2f}°")
    print(f"  Landscape Position: {landscape_data_clean['landscape_position']}")
    
    if rock_data.get('soil_depth_cm'):
        print(f"\nSoil Depth: {rock_data['soil_depth_cm']:.1f} cm")
        if rock_data.get('soil_depth_min_cm') and rock_data.get('soil_depth_max_cm'):
            print(f"  Soil depth range (1km): {rock_data['soil_depth_min_cm']:.1f} - {rock_data['soil_depth_max_cm']:.1f} cm")
        print(f"  Rock Classification: {rock_data['rock_classification']}")
        print(f"  Rock outcrop probability (1km): {rock_data.get('rock_outcrop_probability', 'N/A'):.2f}")
    
    if soil_moisture_data.get('soil_moisture_mean_m3m3'):
        print(f"\nSoil Moisture Summary:")
        print(f"  Mean: {soil_moisture_data['soil_moisture_mean_m3m3']:.4f} m³/m³")
        if soil_moisture_data.get('soil_moisture_mean_min_m3m3') is not None:
            print(f"  Range: {soil_moisture_data['soil_moisture_mean_min_m3m3']:.4f} - {soil_moisture_data['soil_moisture_mean_max_m3m3']:.4f} m³/m³")
        monthly_count = len(soil_moisture_data.get('soil_moisture_monthly', {}))
        if monthly_count > 0:
            print(f"  Monthly time series: {monthly_count // 3} months (surface, rootzone, mean)")
    
    print(f"\nTotal Data Points: {len(df)}")
    print("="*70)
    
    return df

results_df = compile_and_export_results(COORDINATE_STR, LATITUDE, LONGITUDE, topographic_data, soil_data, landscape_data, rock_data, soil_moisture_data, OUTPUT_DIR)


Compiling Results

✓ CSV file saved: -31.60_117.50_soils_profile.csv
  Format: 2 columns (Data_Title, Value)
  Rows: 47
✓ JSON file saved: -31.60_117.50_soils_profile.json

Extraction Summary
Point: (-31.600000, 117.500000)

Topographic:
  Elevation: 257.00 m
  Elevation range (1km): 248.00 - 289.00 m
  Slope: 0.93°
  Aspect: 0.00°
  Landscape Position: Mid Slope

Soil Depth: 148.3 cm
  Soil depth range (1km): 132.5 - 169.6 cm
  Rock Classification: Soil Present
  Rock outcrop probability (1km): 0.00

Soil Moisture: 0.1751 m³/m³

Total Data Points: 47


## Section 10: Results Display

Display the extracted data in a formatted table.

In [ ]:
print("\n" + "="*70)
print("Extracted Data Summary")
print("="*70)
print(f"\nPoint: ({LATITUDE:.6f}, {LONGITUDE:.6f})")
print(f"Output files saved to: {OUTPUT_DIR}")

pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', 50)

print("\nResults DataFrame:")
print(results_df.to_string(index=False))

print("\n" + "="*70)
print("Extraction Complete!")
print("="*70)


Extracted Data Summary

Point: (-31.600000, 117.500000)
Output files saved to: C:\Users\ibian\Desktop\ClimAdapt\Anameka\Anameka_Soils_Profile\TS2 Central

Results DataFrame:
                 Data_Title               Value
                   point_id       -31.60_117.50
                   latitude               -31.6
                  longitude               117.5
            extraction_date 2026-02-09 16:31:14
                elevation_m                 257
        elevation_min_1km_m                 248
        elevation_max_1km_m                 289
              slope_degrees             0.92741
             aspect_degrees                   0
                        twi            8.606927
         landscape_position           Mid Slope
        local_elevation_min                 257
        local_elevation_max                 257
       local_elevation_mean                 257
   rock_outcrop_probability                   0
               bedrock_high               False
         